In [3]:
import collections
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
import pandas as pd

In [4]:
df = pd.read_csv('Final sample Gettech.csv',sep=';',index_col=0)
#df.head()

In [5]:
'''Combining ids'''

df_test = df.groupby(['accommodation_id','basename','at','description','value_type_id'])['amenities_id'].apply(list)
df_test = df_test.to_frame().reset_index()
#df_test.head()

In [6]:
'''Combining contents'''

df_test2 = df.groupby(['accommodation_id','basename','at','description','value_type_id'])['amenities_cont'].apply(list)
df_test2 = df_test2.to_frame().reset_index()
#df_test2.head()

In [11]:
final_df = pd.merge(df_test, df_test2[["accommodation_id", "amenities_cont"]], on="accommodation_id", how="left")
final_df.to_csv('processed_csv_file.csv', sep='\t', encoding='utf-8')
final_df.head()

,accommodation_id,basename,at,description,value_type_id,amenities_id,amenities_cont
0,5088,Hotel Sonne,2,"Set in a prime location of Waldbronn, Hotel So...",1,[47],[1]
1,5092,Steuermann,2,Ideally located in the prime touristic area of...,1,"[63, 47]","[1, 1]"
2,5323,Hotel Montree,2,Hotel Montree is conveniently located in the p...,1,"[63, 48]","[1, 1]"
3,5434,Weidenhof,2,The Weidenhof is a well-situated Hotel in Rege...,1,[48],[1]
4,5668,East-Side,3,This guest house is located in Berlin’s trendy...,1,[255],[1]


In [13]:
item_names = final_df['basename'].tolist()
item_descriptions = final_df['description'].tolist()
print(item_names[:5])
print(item_descriptions[:5])

['Hotel Sonne', 'Steuermann', 'Hotel Montree', 'Weidenhof', 'East-Side']
['Set in a prime location of Waldbronn, Hotel Sonne puts everything the city has to offer just outside', 'Ideally located in the prime touristic area of Karlsruhe City Center, Steuermann promises a relaxing', 'Hotel Montree is conveniently located in the popular Maxvorstadt area. Both business travelers and t', 'The Weidenhof is a well-situated Hotel in Regensburg, Germany.\n\nFor travellers, parking amenities includes a close by car park. Wireless Internet access is available throughout the hotel in rooms only.\n\nRooms within this accommodation feature facilities such as sound proof windows, cosmetic mirror, hairdryer and windows that open. In-room entertainment is available for visitors, which is provided by pay TV.\n\n', 'This guest house is located in Berlin’s trendy going-out Prenzlauer Berg district. The guest house is 250 metres from Eberswalder Straße Underground Station. Eastside Pension is a classic Ber

In [8]:
hotel_descriptions = final_df['description'].tolist()
print(len(hotel_descriptions))

6000


In [9]:
def word_tokenizer(text):
    #tokenizes and stems the text
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
    return tokens


def cluster_sentences(sentences, nb_of_clusters=5):
    stopWords = stopwords.words('english')
    stopWords.extend(stopwords.words('spanish'))
    stopWords.extend(stopwords.words('french'))
    #stopWords.union(["La", "Las","la"])
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                    stop_words=stopWords,
                                    lowercase=True)
    #builds a tf-idf matrix for the sentences
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    kmeans = KMeans(n_clusters=nb_of_clusters)
    kmeans.fit(tfidf_matrix)
    clusters = collections.defaultdict(list)
    for i, label in enumerate(kmeans.labels_):
            clusters[label].append(i)
    return dict(clusters),kmeans,tfidf_vectorizer


if __name__ == "__main__":
    clust_dict = {}
    sentences = hotel_descriptions
    nclusters= 2
    clusters,model,vectorizer = cluster_sentences(sentences, nclusters)
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    '''#print(type(terms))
    for i in range(nclusters):
        print("Cluster %d:" % i)
        for ind in order_centroids[i, :100]:
            print(' %s' % terms[ind])
    for cluster in range(nclusters):
        #print("cluster ",cluster,":")
        items = []
        for i,sentence in enumerate(clusters[cluster]):
                #print("\titem name : ",sentences[sentence])
                items.append(sentences[sentence])
        clust_dict[cluster] = items
    #print(clust_dict)'''

C:\Users\Barshana\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'alguna', 'alguno', 'ant', 'aurion', 'auron', 'avai', 'avion', 'avon', 'ayon', 'could', 'dan', 'desd', 'dond', 'durant', 'ell', 'ello', 'entr', 'erai', 'ere', 'estabai', 'estamo', 'estaremo', 'estaréi', 'estaríai', 'estaríamo', 'estemo', 'estuv', 'estuvierai', 'estuvies', 'estuviesei', 'estuvimo', 'estuvist', 'estuvistei', 'estuviéramo', 'estuviésemo', 'estábamo', 'estái', 'estéi', 'euss', 'eussion', 'eûm', 'fu', 'fuerai', 'fues', 'fuesei', 'fuimo', 'fuist', 'fuistei', 'fuss', 'fussion', 'fuéramo', 'fuésemo', 'fûme', 'fûte', 'habremo', 'habréi', 'habríai', 'habríamo', 'habéi', 'habíai', 'habíamo', 'hayamo', 'hayái', 'hemo', 'hubierai', 'hubies', 'hubiesei', 'hubimo', 'hubist', 'hubistei', 'hubiéramo', 'hubiésemo', 'mai', 'might', 'must', 'má',

<class 'list'>
Cluster 0:
 locat
 hotel
 whether
 prime
 popular
 offer
 busi
 conveni
 ,
 .
 wonder
 everyth
 put
 explor
 area
 're
 perfectli
 citi
 discov
 amongst
 promis
 stop
 leisur
 set
 comfort
 travel
 ideal
 choic
 tourist
 inn
 holiday
 guest
 relax
 3-star
 point
 start
 perfect
 (
 )
 resort
 &
 suit
 pass
 properti
 rang
 wide
 2-star
 featur
 outsid
 ju
 -
 varieti
 villa
 4-star
 high
 center
 hous
 apart
 motel
 complet
 pa
 list
 lodg
 beach
 super
 grand
 visit
 resid
 b
 island
 san
 wond
 hampton
 th
 boutiqu
 8
 h
 hostel
 spa
 facil
 ho
 casa
 qualiti
 p
 plaza
 standard
 bay
 hôtel
 day
 busin
 holida
 ca
 ''
 airport
 istanbul
 america
 wonderf
 best
 hilton
 w
Cluster 1:
 .
 ,
 hotel
 room
 park
 includ
 free
 guest
 avail
 featur
 amen
 restaur
 facil
 offer
 access
 accommod
 servic
 also
 area
 locat
 pool
 tv
 complimentari
 provid
 enjoy
 away
 breakfast
 desk
 beach
 car
 internet
 within
 )
 (
 on-sit
 minut
 ’
 walk
 apart
 privat
 properti
 km
 cent

In [ ]:

films = { 'title': item, 'rank': ranks, 'synopsis': synopses, 'cluster': clusters, 'genre': genres }

frame = pd.DataFrame(films, index = [clusters] , columns = ['rank', 'title', 'cluster', 'genre'])

In [ ]:
# -*- coding: utf-8 -*-
"""
@author: maya_he
"""
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import random

d= {}

#Function for black & white image of Word cloud
def grey_color_func(word, font_size, position, orientation, random_state=None,
                    **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

#Generating wordcloud
stopWords = stopwords.words('english')
stopWords.extend(stopwords.words('spanish'))
stopWords.extend(stopwords.words('french'))
swords = set(stopWords)
wc = WordCloud(width=900,height=500, max_words=200,stopwords=swords,normalize_plurals=False)
for cluster in clust_dict:
    wc.generate((" ").join(clust_dict[cluster]))
    wc.recolor(color_func=grey_color_func,random_state=3)
    #Store the output as a png
    #wc.to_file("wc.png")
    #Plotting the Image
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.show()

    print("----- WordCloud stored as PNG Successfully -----")

In [ ]:
li = ['Tainan House E', 'Vityaz House', 'OYO 14275 Markynti Guest House', 'Belmont House', 'Trafalgar House', 'Becket House', 'Tavira House', 'The Edenfield Guest House', 'Henrietta House', 'New Guest House', 'Arundel House', 'Katti House', 'Murray House', 'Homey - Guest House', 'Evergreen Guest House', 'Evergreen Garden House', 'Balmore House', 'College House', 'Clareview House', 'Jardinets Guest House', 'Highways House', 'Chestnuts House', 'Guest House on Ulitsa Uchitelskaya', 'Guest House Hristina', 'Sagar Belaya Guest House', 'Ariel Guest House', 'Guest House Andrea', 'Guest House Apartments Makkah', 'Osmanbey Cave House', 'Golden Guest House', 'Guest House Florencia', "'t Sand Guest House", 'Marbia Guest House', 'Guest House Karavella', 'Genius Guest House', 'Merryland Guest House', 'Mount Zion Guest House', 'G House', 'Shangrila Guest House', 'Min House Camp', 'Guest House Viktoria', 'Guest House Tatyana', 'Aliens Guest House', 'Mini House', 'Hotel Ohio House Motel', 'Summer House', 'Aist Guest House', 'Altair Guest House', 'Viktoria Guest House', 'Hop Art House Serviced', 'Orangerie Guest House', 'Sairu Guest House', 'Country House San Potente Assisi', 'Il Maggese House', 'Dom & House - Apartamenty Zacisze', 'Wayfarer Guest House', "Yellow House - Holiday's House", 'Larnaka Beach House', 'Bamboo Banks Farm And Guest House', 'El Guembe Hostel House', 'Iguassu Guest House', 'TGH the Guest House Badian', 'Hakodateyama Guest House', 'Art House Hostel On Polyanka', 'Bamber House', 'Peniche Beach House', "Pinar's Bungalow Houses", 'Registhan Guest House', 'Blanc Guest House', 'Plaza Spain Guest House', 'Il Moro Country House', 'Le Vignole Country House', 'Sicily Country House & Beach', 'La Mer Hotel & Dewey House', 'Flag House Resort', 'Summer House on Romar Beach by Wyndham Vacation Rentals', 'Casa Bonita Guest House', 'Geriba House', 'Estaleiro Guest House']

In [ ]:
print(len(li))